#### Metadata
- Link to website: https://www.imdb.com/search/title/?title_type=feature&year=1950-01-01,2012-12-31&sort=alpha,asc
- how to get source code of website in python?

## can you scrap forums on hello iitk

In [1]:
import time
import requests
import json
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tn
from lxml import html
import pandas as pd
import numpy as np

- Keep in mind this link is alphabatically sorted or whaterver see yourself

In [2]:
web_link='https://www.imdb.com/search/title/?title_type=feature&year=1950-01-01,2012-12-31&sort=boxoffice_gross_us,asc'
imdb_data=requests.get(web_link)
if imdb_data.status_code==200:
    print("Status is green 💚")
else:
    print("Status is red 🍎")

soup=bs(imdb_data.text,features='html.parser')
#print(soup.prettify())

# getting this donw
film_name=[] #done
film_image_link=[] #done
film_link=[] #done
film_rating=[] #done
film_votes=[] #done
film_gross_income=[] #done
film_director=[]
film_cast=[]
film_metascore=[] #done
release_year=[] #done
film_length=[] #done
film_geners=[] #done

print('Carefull: each page will take about 5 seconds to process')
how_many_pages=input('How many pages do you want to process? ')
print('ETA:',5*int(how_many_pages),'seconds.')

time1=time.time()

page_counter=0
while(soup.find_all('div',{'class':'desc'})[0].find('a',{'class':'lister-page-next next-page'})!=None and page_counter<int(how_many_pages)):
    start_time=time.time()
    print(page_counter+1,')   Page No. --> ',str(page_counter+1),' <-- is being processed ...') #to keep track of loops
    for link in soup.find_all('div',{'class':'lister-item mode-advanced'}):

        # image link
        film_image_link.append('https://www.imdb.com'+link.find('div',{'class':'lister-item-image float-left'}).a.get('href'))
        foo1=link.find('div',{'class':'lister-item-content'})

        # name, link, realease year
        foo2=foo1.find('h3',{'class':'lister-item-header'})
        film_link.append('https://www.imdb.com'+foo2.a.get('href')) #I am only adding imdb.com/ if link does not work edit it
        film_name.append(foo2.a.contents[0].strip('"'))
        release_year.append(foo2.find('span' ,{'class':"lister-item-year text-muted unbold"}).string[1:5]) # to get rid of '(----)'

        # metascore and imdb rating
        if foo1.find('div',{'class':"inline-block ratings-imdb-rating"})!=None:
            film_rating.append(float(foo1.find('div',{'class':"inline-block ratings-imdb-rating"}).strong.string))
        else:
            film_rating.append(np.nan)
        #film_rating.append(float(foo1.find('div',{'class':"inline-block ratings-imdb-rating"}).strong.string)) # see if it works
        if foo1.find('div',{'class':"inline-block ratings-metascore"})!=None:
            film_metascore.append(foo1.find('div',{'class':"inline-block ratings-metascore"}).span.string)
        else:
            film_metascore.append(np.nan)

        # length, geners
        foo5=foo1.find('p',{'class':"text-muted"})
        how_many_geners=len(foo5.contents)
        if how_many_geners>1:
            if foo5.find('span',{'class':'genre'})!=None:
                if foo5.find('span',{'class':'runtime'})!=None:
                    film_geners.append(foo5.find('span',{'class':'genre'}).string.strip())
                    film_length.append(foo5.find('span',{'class':'runtime'}).string)
                else:
                    film_geners.append(foo5.find('span',{'class':'genre'}).string.strip())
                    film_length.append(np.nan)
            else:
                film_geners.append('notFound')
                film_length.append(np.nan)
        else:
            film_geners.append('notFound')
            film_length.append(np.nan)

        '''# director and cast
        ## task is to define a func. that takes raw list and gives stars and direcctors 
        ## OR takes names without '\n' and then gives stars and directors
        foo4=foo1.find('p',{'class':""})
        how_many_stars=len(foo4.contents)
        if how_many_stars>1:

        else:
            film_director.append('notFound')
            film_cast.append('notFound')'''

        # votes and gross boxoffice
        ## check if votes are available or not
        foo3=foo1.find('p',{'class':"sort-num_votes-visible"})
        if foo3!=None:
            ## check is gross income is there or not :::: but no need to check reverse IG
            if len(foo1.find_all('span',{'name':'nv'}))>1:
                film_votes.append(foo3.find_all('span',{'name':'nv'})[0].string)
                film_gross_income.append(foo3.find_all('span',{'name':'nv'})[1].string)
            else:
                film_votes.append(foo3.find_all('span',{'name':'nv'})[0].string)
                film_gross_income.append(np.nan)

        else:
            film_gross_income.append(np.nan)
            film_votes.append(np.nan)
            
    next_page_link='https://www.imdb.com'+soup.find_all('div',{'class':'desc'})[0].find('a',{'class':'lister-page-next next-page'}).get('href')
    imdb_data=requests.get(next_page_link)
    soup=bs(imdb_data.text,features='html.parser')
    page_counter+=1
    
    # to get how much time it takes
    end_time=time.time()
    time_taken=end_time-start_time
    print('Completed in 👉',round(time_taken,1),'seconds.')# as the all are taking time in seconds ig
    
time2=time.time()
print('👉 It took us',round(time2-time1,1),'seconds, to process your request of processing', how_many_pages,'pages from IMDB webpage. 👈')

Status is green 💚
Carefull: each page will take about 5 seconds to process
How many pages do you want to process? 2
ETA: 10 seconds.
1 )   Page No. -->  1  <-- is being processed ...
Completed in 👉 9.5 seconds.
2 )   Page No. -->  2  <-- is being processed ...
Completed in 👉 11.1 seconds.
👉 It took us 20.6 seconds, to process your request of processing 2 pages from IMDB webpage. 👈


In [3]:
imdb_df=pd.DataFrame({
    'Name':film_name,
    'Rating':film_rating,
    'BoxOffice Collection':film_gross_income,
    'Release Year':release_year,
    'Votes':film_votes,
    'Genres':film_geners,
    'Runtime':film_length,
    'Metascore':film_metascore,
    'Page Link':film_link,
    'Image Link':film_image_link
})

In [4]:
imdb_df.isnull().sum()

Name                     0
Rating                   4
BoxOffice Collection     4
Release Year             0
Votes                    0
Genres                   0
Runtime                  6
Metascore               66
Page Link                0
Image Link               0
dtype: int64

**Pointers:**
- BoxOffice Collection is really not there in most of the case when movies are in alphabate order.
- Also metasocre is NaN in most rows.
- Basic point scrapping does give you data but it is also very bad data with lots of BS. specially in metascore and Gross Collection